In [ ]:
import sys
import os
sys.path.append('..')
from mpp.python.mppy import mpp
import re
from concurrent.futures import ThreadPoolExecutor

In [ ]:
runs = []

In [ ]:
mpp.mute = False
mpp.build()

In [ ]:
class Run:
    data = property()
    def __init__(self, kwargs, log):
        self.kwargs = kwargs
        self.log = log
        
    @data.getter
    def data(self):
        log = self.log
        try:
            seconds = float(re.findall("end program after (.*) seconds",log)[0])
        except:
            times = re.findall("end program after (.*):(.*)\..* minutes",log)[0]
            seconds = int(times[0])*60 + int(times[1])
        data = {
            "seconds" : seconds,
        }
        return data

In [ ]:
def runit(procs, kwargs):
    code = mpp.run(procs, "fgmres", kwargs)
    print(code)
    log = "".join(mpp.read_log())
    r = Run(kwargs, log)
    runs.append(r)
    return r

In [ ]:
kwargs =    {"space_deg":1, 
              "time_deg":1, 
              "fgmres":1, 
              "Presmoothing_space":1,
              "Postsmoothing_space":1,
              "Presmoothing_time":10,
              "Postsmoothing_time":10, 
              "LinearSolver":"GMRES",
              "Preconditioner2":"none"
             }
runs.append(runit(32,kwargs))

In [ ]:

for smoothing in range(2,11,3):
    for fgmres in range(1,10):
        kwargs = {"space_deg":2, 
                  "time_deg":2, 
                  "fgmres":fgmres, 
                  "Presmoothing_space":1,
                  "Postsmoothing_space":1,
                  "Presmoothing_time":smoothing,
                  "Postsmoothing_time":smoothing             
                 }

        r = runit(64, kwargs)
        try:
            print(r.data)
        except: pass

In [ ]:
print("fgmres".center(10), "preSmoothTime".center(15),"postSmoothTime".center(15), "time".center(7))
for r in sorted(runs,key=lambda x:x.data["seconds"]):
    print(str(r.kwargs["fgmres"]).center(10), 
          str(r.kwargs["Presmoothing_time"]).center(15), 
          str(r.kwargs["Postsmoothing_time"]).center(15),
          str(r.data["seconds"]).center(7))

In [ ]:
print("..")


In [ ]:
runs[-1].log